# 0. Todo
- Hello World(タスクとフローの理解)
- タスクに対してのパラメータ化
- 依存タスクの直列実行
- 動的FLow作成(リストの値を渡して、各要素ごとに実行)
- タスクの並列実行
- 条件分岐(前のタスクの結果に応じて次のタスクを分岐)

# 1. Flow(公式チュートリアル)

## 1.1. First Flow

In [1]:
import httpx
from prefect import flow

@flow
def get_repo_info():
    url = "https://api.github.com/repos/PrefectHQ/prefect"
    response = httpx.get(url)
    response.raise_for_status()
    repo = response.json()
    print(f"PrefectHQ/prefect repository statistics 🤓:")
    print(f"Stars 🌠 : {repo['stargazers_count']}")
    print(f"Forks 🍴 : {repo['forks_count']}")


if __name__ == "__main__":
    get_repo_info()

13:13:36.844 | INFO    | prefect.engine - Created flow run 'vigorous-hawk' for flow 'get-repo-info'

PrefectHQ/prefect repository statistics 🤓:
Stars 🌠 : 12245
Forks 🍴 : 1270


13:13:37.448 | INFO    | Flow run 'vigorous-hawk' - Finished in state Completed()

## 1.2. Loggingの使用

In [2]:
# Loggingの追加
# print文を情報ログとして取り込むこともできます（例：@flow(log_prints=True)）。

import httpx
from prefect import flow, get_run_logger


@flow
def get_repo_info(repo_name: str = "PrefectHQ/prefect"):
    url = f"https://api.github.com/repos/{repo_name}"
    response = httpx.get(url)
    response.raise_for_status()
    repo = response.json()
    logger = get_run_logger()
    logger.info(f"PrefectHQ/prefect repository statistics 🤓:")
    logger.info(f"Stars 🌠 : {repo['stargazers_count']}")
    logger.info(f"Forks 🍴 : {repo['forks_count']}")


if __name__ == "__main__":
    get_repo_info()


/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/flows.py:244: UserWarning: A flow named 'get-repo-info' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_56442/661357173.py:6' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


13:13:40.635 | INFO    | prefect.engine - Created flow run 'pristine-skua' for flow 'get-repo-info'

13:13:40.761 | INFO    | Flow run 'pristine-skua' - PrefectHQ/prefect repository statistics 🤓:

13:13:40.763 | INFO    | Flow run 'pristine-skua' - Stars 🌠 : 12245

13:13:40.764 | INFO    | Flow run 'pristine-skua' - Forks 🍴 : 1270

13:13:40.787 | INFO    | Flow run 'pristine-skua' - Finished in state Completed()

## 1.3. リトライ

In [3]:
import httpx
from prefect import flow, get_run_logger


@flow(retries=3, retry_delay_seconds=5)
def get_repo_info(repo_name: str = "PrefectHQ/prefect"):
    url = f"https://api.github.com/repos/{repo_name}"
    response = httpx.get(url)
    response.raise_for_status()
    repo = response.json()
    logger = get_run_logger()
    logger.info(f"PrefectHQ/prefect repository statistics 🤓:")
    logger.info(f"Stars 🌠 : {repo['stargazers_count']}")
    logger.info(f"Forks 🍴 : {repo['forks_count']}")


if __name__ == "__main__":
    get_repo_info()


/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/flows.py:244: UserWarning: A flow named 'get-repo-info' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_56442/3559605385.py:5' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


13:16:58.354 | INFO    | prefect.engine - Created flow run 'talented-mule' for flow 'get-repo-info'

13:16:58.838 | INFO    | Flow run 'talented-mule' - PrefectHQ/prefect repository statistics 🤓:

13:16:58.843 | INFO    | Flow run 'talented-mule' - Stars 🌠 : 12245

13:16:58.846 | INFO    | Flow run 'talented-mule' - Forks 🍴 : 1270

13:16:58.882 | INFO    | Flow run 'talented-mule' - Finished in state Completed()

# 2. Task(公式チュートリアル)

* 注意
  * すべてのタスクはフロー内から呼び出されなければならない。
  * タスクは他のタスクを直接呼び出すことはできません。

## 2.1. taskの使用

In [18]:
# 1のフローをtaskを使用する形にリファクタ
import httpx
from prefect import flow, task, get_run_logger


@task(name="get response task")
def get_url(url: str, params: dict = None):
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return response.json()


@flow(name="get repo info flow")
def get_repo_info(
    repo_name: str = "PrefectHQ/prefect", retries=3, retry_delay_seconds=5
):
    repo = get_url(f"https://api.github.com/repos/{repo_name}")
    logger = get_run_logger()
    logger.info(f"PrefectHQ/prefect repository statistics 🤓:")
    logger.info(f"Stars 🌠 : {repo['stargazers_count']}")
    logger.info(f"Forks 🍴 : {repo['forks_count']}")


if __name__ == "__main__":
    get_repo_info()


16:07:48.209 | INFO    | prefect.engine - Created flow run 'ludicrous-beetle' for flow 'get repo info flow'

16:07:48.296 | INFO    | Flow run 'ludicrous-beetle' - Created task run 'get response task-0' for task 'get response task'

16:07:48.297 | INFO    | Flow run 'ludicrous-beetle' - Executing 'get response task-0' immediately...

16:07:48.655 | INFO    | Task run 'get response task-0' - Finished in state Completed()

16:07:48.656 | INFO    | Flow run 'ludicrous-beetle' - PrefectHQ/prefect repository statistics 🤓:

16:07:48.657 | INFO    | Flow run 'ludicrous-beetle' - Stars 🌠 : 12246

16:07:48.657 | INFO    | Flow run 'ludicrous-beetle' - Forks 🍴 : 1271

16:07:48.674 | INFO    | Flow run 'ludicrous-beetle' - Finished in state Completed('All states completed.')

## 2.2. キャッシュ

* タスクは、戻り値のキャッシュが可能
* ユースケース：
  * フローの実行のたびに取得し直すのが面倒なタスクの結果を保持
  * タスクへの入力が変更されていない場合に、キャッシュされた結果を再利用
* キャッシュを有効にするには、```cache_key_fn```を指定

In [21]:
import httpx
from datetime import timedelta
from prefect import flow, task, get_run_logger
from prefect.tasks import task_input_hash


@task(name="get response task", cache_key_fn=task_input_hash, cache_expiration=timedelta(minutes=1))
def get_url(url: str, params: dict = None):
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return response.json()

In [22]:
@flow(name="get repo info flow")
def get_repo_info(
    repo_name: str = "PrefectHQ/prefect", retries=3, retry_delay_seconds=5
):
    repo = get_url(f"https://api.github.com/repos/{repo_name}")
    logger = get_run_logger()
    logger.info(f"PrefectHQ/prefect repository statistics 🤓:")
    logger.info(f"Stars 🌠 : {repo['stargazers_count']}")
    logger.info(f"Forks 🍴 : {repo['forks_count']}")


if __name__ == "__main__":
    get_repo_info()


/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/flows.py:244: UserWarning: A flow named 'get repo info flow' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_56442/3320423673.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


16:09:51.675 | INFO    | prefect.engine - Created flow run 'warm-toad' for flow 'get repo info flow'

16:09:51.782 | INFO    | Flow run 'warm-toad' - Created task run 'get response task-0' for task 'get response task'

16:09:51.783 | INFO    | Flow run 'warm-toad' - Executing 'get response task-0' immediately...

16:09:52.200 | INFO    | Task run 'get response task-0' - Finished in state Completed()

16:09:52.202 | INFO    | Flow run 'warm-toad' - PrefectHQ/prefect repository statistics 🤓:

16:09:52.203 | INFO    | Flow run 'warm-toad' - Stars 🌠 : 12246

16:09:52.205 | INFO    | Flow run 'warm-toad' - Forks 🍴 : 1271

16:09:52.228 | INFO    | Flow run 'warm-toad' - Finished in state Completed('All states completed.')

## 2.3. 並列処理(concurrency)

In [14]:
import httpx
from datetime import timedelta
from prefect import flow, task, get_run_logger
from prefect.tasks import task_input_hash


@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(minutes=1))
def get_url(url: str, params: dict = None):
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return response.json()


def get_open_issues(repo_name: str, open_issues_count: int, per_page: int = 100):
    issues = []
    pages = range(1, -(open_issues_count // -per_page) + 1)
    for page in pages:
        issues.append(
            get_url(
                f"https://api.github.com/repos/{repo_name}/issues",
                params={"page": page, "per_page": per_page, "state": "open"},
            )
        )
    return [i for p in issues for i in p]


@flow
def get_repo_info(
    repo_name: str = "PrefectHQ/prefect", retries=3, retry_delay_seconds=5
):
    repo = get_url(f"https://api.github.com/repos/{repo_name}")
    issues = get_open_issues(repo_name, repo["open_issues_count"])
    issues_per_user = len(issues) / len(set([i["user"]["id"] for i in issues]))
    logger = get_run_logger()
    logger.info(f"PrefectHQ/prefect repository statistics 🤓:")
    logger.info(f"Stars 🌠 : {repo['stargazers_count']}")
    logger.info(f"Forks 🍴 : {repo['forks_count']}")
    logger.info(f"Average open issues per user 💌 : {issues_per_user:.2f}")


if __name__ == "__main__":
    get_repo_info()


/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'get_url' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_56442/2685729675.py:7' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/flows.py:244: UserWarning: A flow named 'get-repo-info' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_56442/2685729675.py:27' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


13:49:52.965 | INFO    | prefect.engine - Created flow run 'woodoo-hyrax' for flow 'get-repo-info'

13:49:53.033 | INFO    | Flow run 'woodoo-hyrax' - Created task run 'get_url-0' for task 'get_url'

13:49:53.034 | INFO    | Flow run 'woodoo-hyrax' - Executing 'get_url-0' immediately...

13:49:53.063 | INFO    | Task run 'get_url-0' - Finished in state Cached(type=COMPLETED)

...

13:49:59.534 | INFO    | Flow run 'woodoo-hyrax' - PrefectHQ/prefect repository statistics 🤓:

13:49:59.536 | INFO    | Flow run 'woodoo-hyrax' - Stars 🌠 : 12246

13:49:59.537 | INFO    | Flow run 'woodoo-hyrax' - Forks 🍴 : 1270

13:49:59.538 | INFO    | Flow run 'woodoo-hyrax' - Average open issues per user 💌 : 2.36

13:49:59.563 | INFO    | Flow run 'woodoo-hyrax' - Finished in state Completed('All states completed.')

逐次実行を同時実行にするには、submitメソッドを使用する必要がある

In [16]:
import httpx
from datetime import timedelta
from prefect import flow, task, get_run_logger
from prefect.tasks import task_input_hash


@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(minutes=1))
def get_url(url: str, params: dict = None):
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return response.json()


def get_open_issues(repo_name: str, open_issues_count: int, per_page: int = 100):
    issues = []
    pages = range(1, -(open_issues_count // -per_page) + 1)
    for page in pages:
        issues.append(
            get_url.submit(
                f"https://api.github.com/repos/{repo_name}/issues",
                params={"page": page, "per_page": per_page, "state": "open"},
            )
        )
    return [i for p in issues for i in p.result()]


@flow
def get_repo_info(
    repo_name: str = "PrefectHQ/prefect", retries=3, retry_delay_seconds=5
):
    repo = get_url(f"https://api.github.com/repos/{repo_name}")
    issues = get_open_issues(repo_name, repo["open_issues_count"])
    issues_per_user = len(issues) / len(set([i["user"]["id"] for i in issues]))
    logger = get_run_logger()
    logger.info(f"PrefectHQ/prefect repository statistics 🤓:")
    logger.info(f"Stars 🌠 : {repo['stargazers_count']}")
    logger.info(f"Forks 🍴 : {repo['forks_count']}")
    logger.info(f"Average open issues per user 💌 : {issues_per_user:.2f}")


if __name__ == "__main__":
    get_repo_info()


/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'get_url' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_56442/133405939.py:7' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/flows.py:244: UserWarning: A flow named 'get-repo-info' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_56442/133405939.py:27' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


13:57:51.702 | INFO    | prefect.engine - Created flow run 'affable-nyala' for flow 'get-repo-info'

13:57:51.771 | INFO    | Flow run 'affable-nyala' - Created task run 'get_url-0' for task 'get_url'

13:57:51.772 | INFO    | Flow run 'affable-nyala' - Executing 'get_url-0' immediately...

13:57:51.800 | INFO    | Task run 'get_url-0' - Finished in state Cached(type=COMPLETED)

...

13:57:52.220 | INFO    | Flow run 'affable-nyala' - PrefectHQ/prefect repository statistics 🤓:

13:57:52.221 | INFO    | Flow run 'affable-nyala' - Stars 🌠 : 12246

13:57:52.221 | INFO    | Flow run 'affable-nyala' - Forks 🍴 : 1270

13:57:52.222 | INFO    | Flow run 'affable-nyala' - Average open issues per user 💌 : 2.36

13:57:52.240 | INFO    | Flow run 'affable-nyala' - Finished in state Completed('All states completed.')

## 2.4. サブフロー

In [17]:
import httpx
from datetime import timedelta
from prefect import flow, task, get_run_logger
from prefect.tasks import task_input_hash


@task(cache_key_fn=task_input_hash, cache_expiration=timedelta(minutes=1))
def get_url(url: str, params: dict = None):
    response = httpx.get(url, params=params)
    response.raise_for_status()
    return response.json()

@flow
def get_open_issues(repo_name: str, open_issues_count: int, per_page: int = 100):
    issues = []
    pages = range(1, -(open_issues_count // -per_page) + 1)
    for page in pages:
        issues.append(
            get_url.submit(
                f"https://api.github.com/repos/{repo_name}/issues",
                params={"page": page, "per_page": per_page, "state": "open"},
            )
        )
    return [i for p in issues for i in p.result()]


@flow
def get_repo_info(
    repo_name: str = "PrefectHQ/prefect", retries=3, retry_delay_seconds=5
):
    repo = get_url(f"https://api.github.com/repos/{repo_name}")
    issues = get_open_issues(repo_name, repo["open_issues_count"])
    issues_per_user = len(issues) / len(set([i["user"]["id"] for i in issues]))
    logger = get_run_logger()
    logger.info(f"PrefectHQ/prefect repository statistics 🤓:")
    logger.info(f"Stars 🌠 : {repo['stargazers_count']}")
    logger.info(f"Forks 🍴 : {repo['forks_count']}")
    logger.info(f"Average open issues per user 💌 : {issues_per_user:.2f}")


if __name__ == "__main__":
    get_repo_info()


/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'get_url' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_56442/1181491467.py:7' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/kohta/workspace/prefect/venv/lib/python3.10/site-packages/prefect/flows.py:244: UserWarning: A flow named 'get-repo-info' and defined at '/var/folders/q5/s5mcbmt95hd29sp3b9xmr2580000gn/T/ipykernel_56442/1181491467.py:27' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


14:02:48.597 | INFO    | prefect.engine - Created flow run 'greedy-teal' for flow 'get-repo-info'

14:02:48.667 | INFO    | Flow run 'greedy-teal' - Created task run 'get_url-0' for task 'get_url'

14:02:48.668 | INFO    | Flow run 'greedy-teal' - Executing 'get_url-0' immediately...

14:02:48.697 | INFO    | Task run 'get_url-0' - Finished in state Cached(type=COMPLETED)

...

14:02:49.493 | INFO    | Flow run 'greedy-teal' - PrefectHQ/prefect repository statistics 🤓:

14:02:49.495 | INFO    | Flow run 'greedy-teal' - Stars 🌠 : 12246

14:02:49.496 | INFO    | Flow run 'greedy-teal' - Forks 🍴 : 1270

14:02:49.496 | INFO    | Flow run 'greedy-teal' - Average open issues per user 💌 : 2.36

14:02:49.513 | INFO    | Flow run 'greedy-teal' - Finished in state Completed('All states completed.')